In [1]:
from pathlib import Path
import glob as gb
import re
import os

In [2]:
ROOT_DIR = Path(".").absolute().parent.parent
MODEL_DIR = ROOT_DIR / "src" / "model"

In [3]:
cpp_files = sorted(list(MODEL_DIR.glob("*.cpp")))

In [4]:
class ComponentTypeInfo:
    def __init__(self, is_simple: bool, simple_return: str = None, func_body: str = None):
        self.is_simple = is_simple
        self.simple_return = simple_return
        self.func_body = func_body

    def __repr__(self):
        if self.is_simple:
            return self.simple_return
        else:
            return "Complex"

    def __str__(self):
        if self.is_simple:
            return self.simple_return
        else:
            return f"Complex: {func_body}"


class ClassInfo:
    def __init__(self, class_name: str):
        self.class_name = class_name
        self.component_type = None
        self.cooling_fuel_types = None
        self.heating_fuel_types = None
        self.appg_heating_fuel_types = None

    @staticmethod
    def header():
        return ["Class_Name", "ComponentType", "coolingFuelTypes", "heatingFuelTypes", "appGHeatingFuelTypes"]

    def __repr__(self):
        return f"{self.class_name},{self.component_type.__repr__()},{self.cooling_fuel_types.__repr__()},{self.heating_fuel_types.__repr__()},{self.appg_heating_fuel_types.__repr__()}"

In [5]:
RE_SIMPLE_COMPTYPE = re.compile(r"return ComponentType::(Cooling|Heating|None);")
RE_SIMPLE_FUELTYPE = re.compile(r"return (.*);")

RE_INSERT_FUELTYPE = re.compile(r"result.insert\(FuelType::(.*?)\);")
RE_INSERT_CONVERTFUELTYPE = re.compile(r"result.insert\(FuelType\(.*?\)\);")

RE_INSERT_FUELTYPE_APPG = re.compile(r"result.insert\(AppGFuelType::(.*?)\);")
RE_INSERT_CONVERTFUELTYPE_APPG = re.compile(r"result.insert\(convertFuelTypeToAppG\(.*?\)\);")


RE_GET_AIRLOOP_HEATING = re.compile(
    r"\s*if \(auto a_ = airLoopHVAC\(\)\) {\s*return a_->heatingFuelTypes\(\);\s*}\s*return {};\s*", re.DOTALL
)
RE_GET_AIRLOOP_COOLING = re.compile(
    r"\s*if \(auto a_ = airLoopHVAC\(\)\) {\s*return a_->coolingFuelTypes\(\);\s*}\s*return {};\s*", re.DOTALL
)
RE_GET_AIRLOOP_APPG = re.compile(
    r"\s*if \(auto a_ = airLoopHVAC\(\)\) {\s*return a_->appGHeatingFuelTypes\(\);\s*}\s*return {};\s*", re.DOTALL
)


RE_GET_PLANTLOOP_HEATING = re.compile(
    r"\s*if \(auto p_ = plantLoop\(\)\) {\s*return p_->heatingFuelTypes\(\);\s*}\s*return {};\s*", re.DOTALL
)
RE_GET_PLANTLOOP_COOLING = re.compile(
    r"\s*if \(auto p_ = plantLoop\(\)\) {\s*return p_->coolingFuelTypes\(\);\s*}\s*return {};\s*", re.DOTALL
)
RE_GET_PLANTLOOP_APPG = re.compile(
    r"\s*if \(auto p_ = plantLoop\(\)\) {\s*return p_->appGHeatingFuelTypes\(\);\s*}\s*return {};\s*", re.DOTALL
)

RE_INSERT_AIRLOOP_HEATING = re.compile(
    r"\s*if \(auto a_ = airLoopHVAC\(\)\) {\s*for \(auto ft : a_->heatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}\s*}",
    re.DOTALL,
)

RE_INSERT_AIRLOOP_COOLING = re.compile(
    r"\s*if \(auto a_ = airLoopHVAC\(\)\) {\s*for \(auto ft : a_->coolingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}\s*}",
    re.DOTALL,
)

RE_INSERT_AIRLOOP_APPG = re.compile(
    r"\s*if \(auto a_ = airLoopHVAC\(\)\) {\s*for \(auto ft : a_->appGHeatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}\s*}",
    re.DOTALL,
)


RE_INSERT_PLANTLOOP_HEATING = re.compile(
    r"\s*if \(auto p_ = plantLoop\(\)\) {\s*for \(auto ft : p_->heatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}\s*}",
    re.DOTALL,
)

RE_INSERT_PLANTLOOP_COOLING = re.compile(
    r"\s*if \(auto p_ = plantLoop\(\)\) {\s*for \(auto ft : p_->coolingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}\s*}",
    re.DOTALL,
)

RE_INSERT_PLANTLOOP_APPG = re.compile(
    r"\s*if \(auto p_ = plantLoop\(\)\) {\s*for \(auto ft : p_->appGHeatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}\s*}",
    re.DOTALL,
)

RE_INSERT_REHEAT_COIL = re.compile(
    r"\s*for \(auto ft : reheatCoil\(\).heatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)

RE_INSERT_REHEAT_COIL_APPG = re.compile(
    r"\s*for \(auto ft : reheatCoil\(\).appGHeatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)


RE_INSERT_HEAT_COIL = re.compile(
    r"\s*for \(auto ft : heatingCoil\(\).heatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)

RE_INSERT_OPT_HEAT_COIL = re.compile(
    r"\s*for \(auto ft : hc_->heatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)

RE_INSERT_HEAT_COIL_APPG = re.compile(
    r"\s*for \(auto ft : heatingCoil\(\).appGHeatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)

RE_INSERT_OPT_HEAT_COIL_APPG = re.compile(
    r"\s*for \(auto ft : hc_->appGHeatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)


RE_INSERT_SUPHC_COIL = re.compile(
    r"\s*for \(auto ft : supplementalHeatingCoil\(\).heatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)

RE_INSERT_OPT_SUPHC_COIL = re.compile(
    r"\s*for \(auto ft : supHC_->heatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)

RE_INSERT_SUPHC_COIL_APPG = re.compile(
    r"\s*for \(auto ft : supplementalHeatingCoil\(\).appGHeatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)

RE_INSERT_OPT_SUPHC_COIL_APPG = re.compile(
    r"\s*for \(auto ft : supHC_->appGHeatingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)


RE_INSERT_COOL_COIL = re.compile(
    r"\s*for \(auto ft : coolingCoil\(\).coolingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)


RE_INSERT_OPT_COOL_COIL = re.compile(
    r"\s*for \(auto ft : cc_->coolingFuelTypes\(\)\) {\s*result.insert\(ft\);\s*}", re.DOTALL
)

In [6]:
def get_func_body(lines, i):
    c = lines[i].count("{") - lines[i].count("}")
    k = i
    while c != 0:
        k += 1
        c += lines[k].count("{") - lines[k].count("}")
    func_body = "\n".join(lines[i : k + 1])
    func_body = func_body[func_body.find("{") + 1 : func_body.rfind("}")].strip()
    return k, func_body

In [7]:
def parse_coolingFuelTypeFuncBod(func_body: str):
    if m := RE_SIMPLE_FUELTYPE.match(func_body):
        fuel_type = m.groups()[0].strip()
        if fuel_type == "{}":
            fuel_type = ""
        elif fuel_type.startswith("{") and fuel_type.endswith("}"):
            fuel_type = fuel_type.replace("FuelType::", "")
            if "," in fuel_type:
                fuel_type = f'"{fuel_type}"'
            else:
                fuel_type = fuel_type[1:-1]
        return ComponentTypeInfo(is_simple=True, simple_return=fuel_type, func_body=func_body)

    if RE_GET_AIRLOOP_COOLING.match(func_body):
        return ComponentTypeInfo(is_simple=True, simple_return="airLoopHVAC_->coolingFuelTypes()", func_body=func_body)

    if RE_GET_PLANTLOOP_COOLING.match(func_body):
        return ComponentTypeInfo(is_simple=True, simple_return="plantLoop_->coolingFuelTypes()", func_body=func_body)

    s = []
    if m := RE_INSERT_FUELTYPE.search(func_body):
        s.append(m.groups()[0])
    if RE_INSERT_CONVERTFUELTYPE.search(func_body):
        s.append("fuelType()")
    if RE_INSERT_AIRLOOP_COOLING.search(func_body):
        s.append("airLoopHVAC_->coolingFuelTypes()")
    if RE_INSERT_PLANTLOOP_COOLING.search(func_body):
        s.append("plantLoop_->coolingFuelTypes()")
    if RE_INSERT_COOL_COIL.search(func_body):
        s.append("cc.coolingFuelTypes()")
    if RE_INSERT_OPT_COOL_COIL.search(func_body):
        s.append("cc_->coolingFuelTypes()")

    if s:
        # print(s)
        return ComponentTypeInfo(is_simple=True, simple_return=" + ".join(s), func_body=func_body)

    return ComponentTypeInfo(is_simple=False, func_body=func_body)

In [8]:
func_body = """
{
      std::set<FuelType> result;
      result.insert(FuelType::Electricity);
      if (auto p_ = plantLoop()) {
        for (auto ft : p_->coolingFuelTypes()) {
          result.insert(ft);
        }
      }
      return {};
    }
"""
func_body = "return {FuelType::Electricity, FuelType::OtherFuel_1};  // TODO: is that right?"
parse_coolingFuelTypeFuncBod(func_body)

"{Electricity, OtherFuel_1}"

In [9]:
def parse_heatingFuelTypeFuncBod(func_body: str):
    if m := RE_SIMPLE_FUELTYPE.match(func_body):
        fuel_type = m.groups()[0].strip()
        if fuel_type == "{}":
            fuel_type = ""
        elif fuel_type.startswith("{") and fuel_type.endswith("}"):
            fuel_type = fuel_type.replace("FuelType::", "")
            if "," in fuel_type:
                fuel_type = f'"{fuel_type}"'
            else:
                fuel_type = fuel_type[1:-1]
        return ComponentTypeInfo(is_simple=True, simple_return=fuel_type)

    if RE_GET_AIRLOOP_HEATING.match(func_body):
        return ComponentTypeInfo(is_simple=True, simple_return="airLoopHVAC_->heatingFuelTypes()")

    if RE_GET_PLANTLOOP_HEATING.match(func_body):
        return ComponentTypeInfo(is_simple=True, simple_return="plantLoop_->heatingFuelTypes()")

    s = []
    if m := RE_INSERT_FUELTYPE.search(func_body):
        s.append(m.groups()[0])
    if RE_INSERT_CONVERTFUELTYPE.search(func_body):
        s.append("fuelType()")

    if RE_INSERT_AIRLOOP_HEATING.search(func_body):
        s.append("airLoopHVAC_->heatingFuelTypes()")
    if RE_INSERT_PLANTLOOP_HEATING.search(func_body):
        s.append("airLoopHVAC_->heatingFuelTypes()")
    if RE_INSERT_HEAT_COIL.search(func_body):
        s.append("hc.heatingFuelTypes()")
    if RE_INSERT_OPT_HEAT_COIL.search(func_body):
        s.append("hc_->heatingFuelTypes()")
    if RE_INSERT_REHEAT_COIL.search(func_body):
        s.append("reheatCoil.heatingFuelTypes()")
    if RE_INSERT_SUPHC_COIL.search(func_body):
        s.append("supHC.heatingFuelTypes()")
    if RE_INSERT_OPT_SUPHC_COIL.search(func_body):
        s.append("supHC_->heatingFuelTypes()")
    if s:
        # print(s)
        return ComponentTypeInfo(is_simple=True, simple_return=" + ".join(s))

    return ComponentTypeInfo(is_simple=False, func_body=func_body)


def parse_appGHeatingFuelTypeFuncBod(func_body: str):
    if m := RE_SIMPLE_FUELTYPE.match(func_body):
        fuel_type = m.groups()[0].strip()
        if fuel_type == "{}":
            fuel_type = ""
        elif fuel_type.startswith("{") and fuel_type.endswith("}"):
            fuel_type = fuel_type.replace("AppGFuelType::", "")
            if "," in fuel_type:
                fuel_type = f'"{fuel_type}"'
            else:
                fuel_type = fuel_type[1:-1]
        return ComponentTypeInfo(is_simple=True, simple_return=fuel_type, func_body=func_body)

    if RE_GET_AIRLOOP_APPG.match(func_body):
        return ComponentTypeInfo(
            is_simple=True, simple_return="airLoopHVAC_->appGHeatingFuelTypes()", func_body=func_body
        )

    if RE_GET_PLANTLOOP_APPG.match(func_body):
        return ComponentTypeInfo(
            is_simple=True, simple_return="plantLoop_->appGHeatingFuelTypes()", func_body=func_body
        )

    s = []

    if m := RE_INSERT_FUELTYPE_APPG.search(func_body):
        s.append(m.groups()[0])
    if RE_INSERT_CONVERTFUELTYPE_APPG.search(func_body):
        s.append("fuelType()")

    if RE_INSERT_AIRLOOP_APPG.search(func_body):
        s.append("airLoopHVAC_->appGHeatingFuelTypes()")
    if RE_INSERT_PLANTLOOP_APPG.search(func_body):
        s.append("airLoopHVAC_->appGHeatingFuelTypes()")
    if RE_INSERT_HEAT_COIL_APPG.search(func_body):
        s.append("hc.appGHeatingFuelTypes()")
    if RE_INSERT_OPT_HEAT_COIL_APPG.search(func_body):
        s.append("hc_->appGHeatingFuelTypes()")
    if RE_INSERT_REHEAT_COIL_APPG.search(func_body):
        s.append("reheatCoil.appGHeatingFuelTypes()")
    if RE_INSERT_SUPHC_COIL_APPG.search(func_body):
        s.append("supHC.appGHeatingFuelTypes()")
    if RE_INSERT_OPT_SUPHC_COIL_APPG.search(func_body):
        s.append("supHC_->appGHeatingFuelTypes()")
    if s:
        # print(s)
        return ComponentTypeInfo(is_simple=True, simple_return=" + ".join(s), func_body=func_body)

    return ComponentTypeInfo(is_simple=False, func_body=func_body)

In [10]:
func_body = """
{
      std::set<FuelType> result;
      if (auto hc_ = heatingCoil()) {
        for (auto ft : hc_->heatingFuelTypes()) {
          result.insert(ft);
        }
      }
      // TODO: is supplemental needed?
      if (auto supHC_ = supplementalHeatingCoil()) {
        for (auto ft : supHC_->heatingFuelTypes()) {
          result.insert(ft);
        }
      }

      return {result.begin(), result.end()};
    }
"""
parse_heatingFuelTypeFuncBod(func_body)

hc_->heatingFuelTypes() + supHC_->heatingFuelTypes()

In [11]:
func_body = """{
      std::set<AppGFuelType> result;
      if (auto hc_ = heatingCoil()) {
        for (auto ft : hc_->appGHeatingFuelTypes()) {
          result.insert(ft);
        }
      }
      // TODO: is supplemental needed?
      if (auto supHC_ = supplementalHeatingCoil()) {
        for (auto ft : supHC_->appGHeatingFuelTypes()) {
          result.insert(ft);
        }
      }
      return {result.begin(), result.end()};
    }"""

func_body = """{
      std::set<AppGFuelType> result;
      for (auto ft : reheatCoil().appGHeatingFuelTypes()) {
        result.insert(ft);
      }
      if (auto a_ = airLoopHVAC()) {
        for (auto ft : a_->appGHeatingFuelTypes()) {
          result.insert(ft);
        }
      }
      return {result.begin(), result.end()};
    }"""
parse_appGHeatingFuelTypeFuncBod(func_body)

airLoopHVAC_->appGHeatingFuelTypes() + reheatCoil.appGHeatingFuelTypes()

In [12]:
class_infos = {}
for cpp_file in cpp_files:
    class_name = cpp_file.name.replace(".cpp", "")

    with open(cpp_file, "r") as f:
        content = f.read()
    if not "Impl::componentType()" in content:
        continue

    lines = content.splitlines()

    class_info = ClassInfo(class_name=class_name)

    i = 0
    while i < len(lines):
        if "::componentType" in lines[i]:
            i, func_body = get_func_body(lines, i)
            if m := RE_SIMPLE_COMPTYPE.match(func_body):
                class_info.component_type = ComponentTypeInfo(is_simple=True, simple_return=m.groups()[0])
            else:
                class_info.component_type = ComponentTypeInfo(is_simple=False, func_body=func_body)
        elif "::coolingFuelTypes" in lines[i]:
            i, func_body = get_func_body(lines, i)
            class_info.cooling_fuel_types = parse_coolingFuelTypeFuncBod(func_body=func_body)

        elif "::heatingFuelTypes" in lines[i]:
            i, func_body = get_func_body(lines, i)
            class_info.heating_fuel_types = parse_heatingFuelTypeFuncBod(func_body=func_body)
        elif "::appGHeatingFuelTypes" in lines[i]:
            i, func_body = get_func_body(lines, i)
            class_info.appg_heating_fuel_types = parse_appGHeatingFuelTypeFuncBod(func_body=func_body)
        i += 1
    class_infos[class_name] = class_info

In [13]:
class_infos["ZoneHVACPackagedTerminalHeatPump"]

ZoneHVACPackagedTerminalHeatPump,Complex,cc.coolingFuelTypes(),hc.heatingFuelTypes() + supHC.heatingFuelTypes(),hc.appGHeatingFuelTypes() + supHC.appGHeatingFuelTypes()

# Print out "complex" types I couldn't resolve

## Cooling fuel types

In [14]:
for k, c in class_infos.items():
    if c.cooling_fuel_types.is_simple:
        continue

    print(f"{c.class_name}\n==============\n{c.cooling_fuel_types.func_body}\n\n")

AirLoopHVACOutdoorAirSystem
std::set<FuelType> result;
      for (const auto& comp : subsetCastVector<HVACComponent>(components())) {
        for (auto& ft : comp.coolingFuelTypes()) {
          result.insert(ft);
        }
      }
      return {result.begin(), result.end()};


ChillerAbsorption
if (auto generatorLoop_ = generatorLoop()) {
        return generatorLoop_->heatingFuelTypes();  // This is done on purpose. We use heat to produce chilled water here!
      }
      return {};


ChillerAbsorptionIndirect
if (auto generatorLoop_ = generatorLoop()) {
        return generatorLoop_->heatingFuelTypes();  // This is done on purpose. We use heat to produce chilled water here!
      }
      return {};


CoilSystemIntegratedHeatPumpAirSource
std::set<FuelType> result;

      for (auto ft : spaceCoolingCoil().coolingFuelTypes()) {
        result.insert(ft);
      }

      for (auto ft : scwhCoil().coolingFuelTypes()) {
        result.insert(ft);
      }
      for (auto ft : scdwhCoolingC

## Heating Fuel Types

In [15]:
for k, c in class_infos.items():
    if c.heating_fuel_types.is_simple:
        continue

    print(f"{c.class_name}\n==============\n{c.heating_fuel_types.func_body}\n\n")

AirLoopHVACOutdoorAirSystem
std::set<FuelType> result;
      for (const auto& comp : subsetCastVector<HVACComponent>(components())) {
        for (auto& ft : comp.heatingFuelTypes()) {
          result.insert(ft);
        }
      }
      return {result.begin(), result.end()};


CoilSystemIntegratedHeatPumpAirSource
std::set<FuelType> result;

      for (auto ft : spaceHeatingCoil().heatingFuelTypes()) {
        result.insert(ft);
      }
      // TODO: include dedicatedWaterHeatingCoil? The combined space cooling + water heating ones too?
      for (auto ft : shdwhHeatingCoil().heatingFuelTypes()) {
        result.insert(ft);
      }
      for (auto ft : shdwhWaterHeatingCoil().heatingFuelTypes()) {
        result.insert(ft);
      }

      return {result.begin(), result.end()};


CoilWaterHeatingAirToWaterHeatPumpWrapped
//   return {FuelType::Electricity};
    //


HeatExchangerFluidToFluid
const std::string controlType = this->controlType();
      if (openstudio::istringEqual(contro

## Appendix G Heating Fuel Types

In [16]:
for k, c in class_infos.items():
    if c.appg_heating_fuel_types.is_simple:
        continue

    print(f"{c.class_name}\n==============\n{c.appg_heating_fuel_types.func_body}\n\n")

AirLoopHVACOutdoorAirSystem
std::set<AppGFuelType> result;
      for (const auto& comp : subsetCastVector<HVACComponent>(components())) {
        for (auto& ft : comp.appGHeatingFuelTypes()) {
          result.insert(ft);
        }
      }
      return {result.begin(), result.end()};


CoilSystemIntegratedHeatPumpAirSource
std::set<AppGFuelType> result;

      for (auto ft : spaceHeatingCoil().appGHeatingFuelTypes()) {
        result.insert(ft);
      }
      // TODO: include dedicatedWaterHeatingCoil? The combined space cooling + water heating ones too?
      for (auto ft : shdwhHeatingCoil().appGHeatingFuelTypes()) {
        result.insert(ft);
      }
      for (auto ft : shdwhWaterHeatingCoil().appGHeatingFuelTypes()) {
        result.insert(ft);
      }

      return {result.begin(), result.end()};


CoilWaterHeatingAirToWaterHeatPumpWrapped
//   return {AppGFuelType::HeatPump};  // TODO: openstudio-standards uses Electric
    //


HeatExchangerFluidToFluid
const std::string contro

# Dump to CSV

In [17]:
lines = [",".join(ClassInfo.header())]
for k, c in class_infos.items():
    lines.append(c.__repr__())

with open("FuelTypes.csv", "w") as f:
    f.write("\n".join(lines) + "\n")